In [1]:
import qsharp
from IPython.display import Latex
import math

In [2]:
def is_significant(x: float):
    return abs(x) > 1e-9

def is_fractional_part_significant(x: float):
    return is_significant(x - round(x))

# Takes float number
# Returns recognized/not, numerator, denominator
# Sign is in the numerator
def recognize_nice_rational(x: float):
    for denominator in range(1, 31):
        # TODO: Optimize this. Consider squares of roots in the denominator.
        numerator: float = x * float(denominator)
        if not is_fractional_part_significant(numerator):
            return True, int(round(numerator)), denominator
    return False, 0, 1

# Takes float number
# Returns recognized/not, numerator, denominator, root
# Sign is in the numerator
roots = [1, 2, 3, 5, 6, 7, 10, 11, 13, 14, 15, 17, 19, 21, 22, 23, 26, 29, 30]
def recognize_nice_algebraic(x: float):
    for root in roots:
        # TODO: Optimize this. In practice we don't really need anything beyond 1, √2, √3, √5, √6.
        by_root = x / math.sqrt(root)
        (is_rational, numerator, denominator) = recognize_nice_rational(by_root)
        if is_rational:
            return True, numerator, denominator, root
    return False, 0, 1, 0

# Takes real and imaginary part of a complex number
# returns recognized/not, magnitude_num, magnitude_den, pi_num, pi_den
# Always returns positive magnitude
# Return angles in [-1, 1] * pi, sign is in the pi_num
pi_fractions = [(1, 3), (2, 3), (1, 4), (3, 4), (1, 8), (3, 8), (5, 8), (7, 8)]
def recognize_nice_exponent(re: float, im: float):
    #print(f"re={re}, im={im}")
    for (pi_num, pi_den) in pi_fractions:
        angle: float = math.pi*float(pi_num)/float(pi_den);
        s: float = math.sin(angle)
        c: float = math.cos(angle)
        if not is_significant(re/c - im/s):
            #print(f"pi_num={pi_num}, pi_den={pi_den}")
            # It's OK to take abs value as we are only interested in magnitude
            (success, mag_num, mag_den) = recognize_nice_rational(abs(re/c))
            #print(f"s={s}, c={c}, success={success}, numerator={magnitude_num}, denominator={magnitude_den}")
            if success:
                if im > 0:
                    return True, mag_num, mag_den, pi_num, pi_den
                else:
                    return True, mag_num, mag_den, pi_num-pi_den, pi_den
    return False, 0, 1, 0, 1

def get_latex_for_decimal(x: float):
    return f"{x}"

def get_latex_for_algebraic(numerator: int, denominator: int, root: int, needs_1: bool):
    if denominator != 1:
        needs_1 = True

    latex = ""
    if root == 1:
        if numerator == 1:
            if needs_1:
                latex = "1"
        else:
            latex = f"{numerator}"
    else:
        if numerator == 1:
            latex = f"\\sqrt{{{root}}}"
        else:
            latex = f"{numerator} \\sqrt{{{root}}}"

    if denominator != 1:
        latex = f"\\frac{{{latex}}}{{{denominator}}}"
    return latex

def get_latex_for_exponent(pi_num: int, pi_den: int):
    latex = "e^{{"
    abs_pi_num: int = pi_num
    if pi_num < 0:
        abs_pi_num = - pi_num
        latex += "-"
    if abs_pi_num != 1:
        latex += f"{abs_pi_num}"
    latex += " i \\pi "
    if pi_den != 1:
        latex += f" / {pi_den}"
    latex += "}}"
    return latex

def get_latex_for_exponent_coefficient(mag_num: int, mag_den: int, pi_num: int, pi_den: int, needs_plus: bool):
    latex = ""
    if needs_plus:
        latex += "+"
    latex += get_latex_for_algebraic(mag_num, mag_den, 1, False)
    latex += get_latex_for_exponent(pi_num, pi_den)
    return latex;
    
def display_state_latex(dump: qsharp.StateDump):
    dict = dump.get_dict()
    rows = list(dict.items())

    state_latex = ""
    term_number = 0
    for row in rows:
        [state, [real, imag]] = row

        real_significant = is_significant(real)
        imag_significant = is_significant(imag)
        if (not real_significant and not imag_significant):
            continue
        term_number += 1

        if real_significant:
            if imag_significant:
                # both real and imaginary amplitude
                (is_exponent, mag_num, mag_den, pi_num, pi_den) = recognize_nice_exponent(real, imag);
                if is_exponent:
                    state_latex += get_latex_for_exponent_coefficient(mag_num, mag_den, pi_num, pi_den, term_number > 1)
                else:
                    (is_algebraic1, numerator1, denominator1, root1) = recognize_nice_algebraic(abs(real))
                    (is_algebraic2, numerator2, denominator2, root2) = recognize_nice_algebraic(abs(imag))
                    if is_algebraic1:

                    else:



                    (is_positive1, latex1) = recognize_nice_algebraic(real, True)
                    (is_positive2, latex2) = recognize_nice_algebraic(imag, False)
                    if not is_positive1:
                        state_latex += "-"
                    elif term_number > 1:
                        state_latex += "+"
                    imag_sign = "+" if is_positive1 == is_positive2 else "-"
                    state_latex += f"\\left( {latex1} {imag_sign} {latex2}i \\right)"
            else:
                # only real amplitude
                (is_algebraic, numerator, denominator, root) = recognize_nice_algebraic(abs(real))
                if is_algebraic:
                    latex = get_latex_for_algebraic(numerator, denominator, root, False)
                else:
                    latex = get_latex_for_decimal(abs(real))
                if real < 0:
                    state_latex += "-"
                elif term_number > 1:
                    state_latex += "+"
                state_latex += f"{latex} \\ "
        else:
            # only imaginary amplitude
            (is_algebraic, numerator, denominator, root) = recognize_nice_algebraic(abs(imag))
            if is_algebraic:
                latex = get_latex_for_algebraic(numerator, denominator, root, False)
            else:
                latex = get_latex_for_decimal(abs(imag))
            if imag < 0:
                state_latex += "-"
            elif term_number > 1:
                state_latex += "+"
            state_latex += f"{latex}i \\ "

        state_latex += f"|\\boldsymbol{{{state}}}\\rangle"
            
    #print(state_latex)
    display(Latex(f"$|\\boldsymbol{{\\psi}}\\rangle = {state_latex}$"))

# TODO
# Normalization coefficient can be common
# State label better be in binary
# Recognize separable states
# Recongnize well-known states: |0> |1> |+> |-> |Bell> |CATn>


In [5]:
%%qsharp
use q = Qubit();
X(q);
T(q);

In [15]:
%%qsharp
Adjoint T(q);

In [4]:
%%qsharp
use q = Qubit();
X(q);
H(q);

In [3]:
%%qsharp

use qs = Qubit[2];
H(qs[0]);
CNOT(qs[0], qs[1]);

In [3]:
%%qsharp

use q = Qubit();
H(q);
X(q);
Z(q);
Y(q);

In [4]:
%%qsharp

use qs = Qubit[2];
H(qs[0]);
H(qs[1]);
S(qs[0]);
T(qs[1]);

// Input: A two-qubit system in the basis state |00>
// Goal: Transform the system into the state
// 1/2 (|00> + e^(i𝜋/4)|01> + e^(i𝜋/2)|10> + e^(3i𝜋/4)|11>)


In [4]:
%%qsharp

use qs = Qubit[2];
H(qs[0])
CNOT(qs[0], qs[1]);

//ResetAll(qs)

X(qs[1]);

X(qs[0]);
//X(qs[1]);
RFrac(PauliZ, 7, 3, qs[0])


In [5]:
%%qsharp
use q = Qubit[7];
H(q[0]);
H(q[1]);
H(q[3]);
CX(q[0], q[2]);
CX(q[0], q[4]);
CX(q[0], q[6]);
CX(q[1], q[2]);
CX(q[1], q[5]);
CX(q[1], q[6]);
CX(q[3], q[4]);
CX(q[3], q[5]);
CX(q[3], q[6]);

In [14]:
%%qsharp
Z(q[0])

In [4]:
dump = qsharp.dump_machine()
display(dump)
display_state_latex(dump)


STATE:
|00⟩: 0.7071+0.0000𝑖
|11⟩: 0.7071+0.0000𝑖

<IPython.core.display.Latex object>